In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

data_dim = 4
timesteps = 50
nb_classes = 2
batch_size = 100
rows = 575
signal_num = [3,4,5,15]

# set up model
model = Sequential()
model.add(LSTM(128, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))

model.add(LSTM(128, stateful=True))


model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using Theano backend.
Using gpu device 0: GeForce GTX 480 (CNMeM is disabled, cuDNN not available)


In [2]:
#get focused data
y_train = np.zeros((rows*2, nb_classes))
classes = np.zeros(rows*2)
focusedData = np.loadtxt("focus_base.csv", delimiter=",", skiprows=1)
t = 0
x = 0
focusedSignals = np.zeros((rows, timesteps, 4))
signal_num = [3,4,5,15]
signal = 0

for n in signal_num:
    t = 0
    x=0
    x_plus=x+timesteps
    while t < rows:
        points = focusedData[x:x_plus, n]
        focusedSignals[t,0:timesteps,signal] = points
        t+=1
        x+=20
        x_plus=x+50
    signal+=1

#set one-hot
for i in range(0,rows):
    y_train[i][0]=1
    y_train[i][1]=0
    classes[i]=1

#get scared data
scaredData = np.loadtxt("scared.txt", delimiter=",", skiprows=1)
t = 0
x = 0

scaredSignals = np.zeros((rows, timesteps, data_dim))
signal_num = [15,4,3,5]
signal = 0

for n in signal_num:
    t = 0
    x = 0
    x_plus=x+timesteps
    while t < rows:
        points = scaredData[x:x_plus, n]
        scaredSignals[t,0:timesteps,signal] = points
        t+=1
        x+=20
        x_plus=x+50
    signal+=1

#set focues one-hot
for i in range(rows, rows*2):
    y_train[i][0]=0
    y_train[i][1]=1
    classes[i]=0
    
signals = np.vstack((focusedSignals,scaredSignals))

In [3]:
#fit
model.fit(signals, y_train, shuffle = True,
          batch_size=batch_size, nb_epoch=200, verbose = 1)

Exception: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 1150 samples